1. Installation des bibliothèques

In [ ]:
import os
import json
from datetime import datetime
import shutil

2. Pipeline normal (sans générateur)

In [ ]:
# Chargement des données
def load_sample(path) -> list:
    """Charge le contenu d'un fichier de transactions en liste de chaînes de caractères"""
    with open(path, 'r') as file:
        return file.readlines()

# Transformation des données vers JSON
def generate_json(transactions: list) -> dict:
    """Transforme les données en un dictionnaire avec nom_emetteur et total_sent"""
    total = 0
    name = ""
    for transaction in transactions:
        data = transaction.split()  # Découpe la ligne "nom_emetteur date montant"
        name = data[0]  # Prend le nom de l'émetteur
        montant = int(data[2].replace("€", ""))  # Transforme le montant en entier
        total += montant  # Somme les montants

    return {"name": name, "total_sent": total}

# Sauvegarde du résultat sous forme de fichier JSON
def save_result(path: str, result: dict):
    """Sauvegarde le résultat sous forme de fichier JSON"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"result_sample_{timestamp}.json"
    filepath = os.path.join(path, filename)

    with open(filepath, 'w') as json_file:
        json.dump(result, json_file, indent=4)

# Pipeline pour process tous les fichiers
def process_all_files(source_dir: str, result_dir: str, archive_dir: str):
    """Process tous les fichiers dans le dossier source, génère les résultats et archive les fichiers"""
    files = os.listdir(source_dir)

    for file in files:
        file_path = os.path.join(source_dir, file)
        # Charger les données
        transactions = load_sample(file_path)
        # Générer le JSON
        result = generate_json(transactions)
        # Sauvegarder le résultat
        save_result(result_dir, result)
        # Archiver le fichier
        archive_file(file_path, archive_dir)

# Fonction pour déplacer les fichiers vers le dossier d'archive
def archive_file(file_path: str, archive_dir: str):
    """Déplace un fichier vers le dossier d'archive"""
    if not os.path.exists(archive_dir):
        os.makedirs(archive_dir)
    shutil.move(file_path, archive_dir)


3. Pipeline avec générateur

In [ ]:
# Connexion à la base de données
conn = mysql.connector.connect(
    host="localhost",  # ou l'adresse IP de ton serveur MySQL
    user="root",       # Nom d'utilisateur MySQL
    password="root",  # Mot de passe MySQL
    database="Ecole"   # Nom de la base de données
)

# Création d'un curseur pour exécuter les requêtes
cursor = conn.cursor()

# 1. Récupérer les données des élèves
cursor.execute("SELECT prenom, nom, numero_classe FROM eleves")
eleves = cursor.fetchall()

# 2. Récupérer les données des enseignants
cursor.execute("SELECT prenom, nom, numero_classe FROM enseignants")
enseignants = cursor.fetchall()

# Fermeture du curseur et de la connexion
cursor.close()
conn.close()

# 3. Associer chaque élève à un enseignant par numéro de classe
for eleve in eleves:
    prenom_eleve, nom_eleve, classe_eleve = eleve
    enseignant_trouve = None

    for enseignant in enseignants:
        prenom_enseignant, nom_enseignant, classe_enseignant = enseignant
        if classe_eleve == classe_enseignant:
            enseignant_trouve = f"{prenom_enseignant} {nom_enseignant}"
            break

    # Afficher l'association élève-enseignant
    if enseignant_trouve:
        print(f"Élève: {prenom_eleve} {nom_eleve}, Classe: {classe_eleve} -> Enseignant responsable: {enseignant_trouve}")
    else:
        print(f"Élève: {prenom_eleve} {nom_eleve}, Classe: {classe_eleve} -> Enseignant responsable: Non trouvé")

4. Exécution du pipeline

In [ ]:
# Directoires pour les fichiers source, résultats et archivage
source_dir = 'source'
result_dir = 'result'
archive_dir = 'archived'

# Création des dossiers si ils n'existent pas
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# Exécution du pipeline normal (sans générateur)
print("Exécution du pipeline sans générateur")
process_all_files(source_dir, result_dir, archive_dir)

# Exécution du pipeline avec générateur
print("Exécution du pipeline avec générateur")
process_all_files_with_generator(source_dir, result_dir, archive_dir)

5. Bonus : Mesurer la consommation mémoire avec memory_profiler

In [16]:
!pip install memory-profiler

from memory_profiler import profile

@profile
def process_all_files_with_generator(source_dir: str, result_dir: str, archive_dir: str):
    files = os.listdir(source_dir)
    for file in files:
        file_path = os.path.join(source_dir, file)
        transactions = load_sample_with_generator(file_path)
        result = generate_json_with_generator(transactions)
        save_result(result_dir, result)
        archive_file(file_path, archive_dir)



Associations élèves-enseignants :
Mark Watney -> Enseignant responsable: Jonas Salk

Nombre d'élèves par enseignant :
Jonas Salk a 1 élève(s).
